<a href="https://colab.research.google.com/github/evaaaaho1999/Text-Mining/blob/main/query_function.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import nltk
import gdown
nltk.download('punkt')
def get_df():
    # df_query
    url = "https://drive.google.com/u/0/uc?id=1-24aWX9oqnhYm65onxV5Zm4PbXppvtGD&export=download"
    gdown.download(url, "news_en.csv", quiet=True)
    return pd.read_csv("news_en.csv")
def get_doc_all(df_query):
    # doc_all
    doc_all={}
    for i in range(len(df_query)):
        tokens = nltk.word_tokenize(df_query.news[i])
        token_filtered = [w.lower() for w in tokens if w.isalpha()]
        doc_all[i]=token_filtered
    return doc_all
# Declare all function 
# create tf function
def tf(term, token_doc):
    tf = token_doc.count(term)/len(token_doc)
    return tf

# create function to calculate how many doc contain the term 
def numDocsContaining(word, token_doclist):
    doccount = 0
    for doc_token in token_doclist:
        if doc_token.count(word) > 0:
            doccount +=1
    return doccount
  
import math
# create function to calculate  Inverse Document Frequency in doclist - this list of all documents
def idf(word, token_doclist):
    n = len(token_doclist)
    df = numDocsContaining(word, token_doclist)
    return math.log10(n/df)

#define a function to do cosine normalization a data dictionary
def cos_norm(dic): # dic is distionary data structure
  import numpy as np
  dic_norm={}
  factor=1.0/np.sqrt(sum([np.square(i) for i in dic.values()]))
  for k in dic:
    dic_norm[k] = dic[k]*factor
  return dic_norm

#create function to calculate normalize tfidf 
def compute_tfidf(token_doc,bag_words_idf):
  tfidf_doc={}
  for word in set(token_doc):
    tfidf_doc[word]= tf(word,token_doc) * bag_words_idf[word]   
  tfidf_norm = cos_norm(tfidf_doc)
  return tfidf_norm

# create normalize term frequency
def tf_norm(token_doc):
  tf_norm={}
  for term in token_doc:
    tf = token_doc.count(term)/len(token_doc)
    tf_norm[term]=tf
  tf_max = max(tf_norm.values())
  for term, value in tf_norm.items():
    tf_norm[term]= 0.5 + 0.5*value/tf_max
  return tf_norm
  
def compute_tfidf_query(query_token,bag_words_idf):
  tfidf_query={}
  tf_norm_query = tf_norm(query_token)
  for term, value in tf_norm_query.items():
    tfidf_query[term]=value*bag_words_idf[term]   
  return tfidf_query
def add_tfidf_of_query(tfidf_query):
    tfidf["query"]
def get_tfidf_query(input_query,bag_words,bag_words_idf):
    # tfidf_query,query_token
    query=str(input_query)
    query=query.lower()
    query_token_raw= nltk.word_tokenize(query)
    query_token = [term for term in query_token_raw if term in bag_words]
    if len(query_token) >0 :
        tfidf_query =compute_tfidf_query(query_token,bag_words_idf) #calculate tfidf for query text
        return tfidf_query,query_token
    else:  
        return 0
def web_crawer():
    import bs4
    from bs4 import BeautifulSoup as soup
    from urllib.request import urlopen

    news_url="https://news.google.com/rss/topics/CAAqJggKIiBDQkFTRWdvSUwyMHZNRGx6TVdZU0FtVnVHZ0pWVXlnQVAB?hl=en-US"
    Client=urlopen(news_url)
    xml_page=Client.read()
    Client.close()

    soup_page=soup(xml_page,"xml")
    news_list=soup_page.findAll("item")
    # Print news title, url and publish date
    news_title=[]
    website=[]
    today_news=[]
    i=0
    for news in news_list:
        news_title.append(news.title.text)
        website.append(news.link.text)
        i+=1
    
    return news_title,website
def query_and_answer(input_query="trump"):
    news_title,website = web_crawer()
    df_query = get_df()
    doc_all = get_doc_all(df_query)
    
    #create bag words
    bag_words =[] # declare bag_words is a list
    for doc in doc_all.keys():
        bag_words += doc_all[doc]
    bag_words=set(bag_words)


    bag_words_idf={} # declare "bag_words_idf" data structure is dictionary 
    bag_words_len = len(bag_words)
    bag_word_10 = round(bag_words_len/10,0)

    i=0
    for word in bag_words:
        i+=1
        bag_words_idf[word]= idf(word,doc_all.values())
    tfidf={} 
    for doc in doc_all.keys():
        tfidf[doc]= compute_tfidf(doc_all[doc],bag_words_idf)

    tfidf_query,query_token = get_tfidf_query(input_query,bag_words,bag_words_idf)

    # add tfidf of query text to tfidf of all doc and convert to dataframe
    tfidf["query"]=tfidf_query

    import pandas as pd
    tfidf_df = pd.DataFrame(tfidf).transpose()
    tfidf_df= tfidf_df.fillna(0) # replace all NaN by zero

    from scipy.spatial.distance import cosine
    cosine_sim ={}
    for row in tfidf_df.index:
        if row != "query":
            cosine_sim[row]= 1-cosine(tfidf_df.loc[row],tfidf_df.loc["query"])

    # the top 10 relevant document
    if len(query_token) >0 :
        cosine_sim_top10 = dict(sorted(cosine_sim.items(), key=lambda item: item[1],reverse=True)[:10])
        # print(cosine_sim_top10)

    returns=list(cosine_sim_top10.keys())[0]

    return (news_title[returns],website[returns])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
tp = query_and_answer("trump")

In [ ]:
tp

('Dallas County Reports Record 328 COVID-19 Cases Friday, Adds 3 Deaths - NBC 5 Dallas-Fort Worth',
 'https://news.google.com/__i/rss/rd/articles/CBMidWh0dHBzOi8vd3d3Lm5iY2Rmdy5jb20vbmV3cy9jb3JvbmF2aXJ1cy9kYWxsYXMtY291bnR5LXJlcG9ydHMtcmVjb3JkLTMyOC1jb3ZpZC0xOS1jYXNlcy1mcmlkYXktYWRkcy0zLWRlYXRocy8yMzg3OTAwL9IBeWh0dHBzOi8vd3d3Lm5iY2Rmdy5jb20vbmV3cy9jb3JvbmF2aXJ1cy9kYWxsYXMtY291bnR5LXJlcG9ydHMtcmVjb3JkLTMyOC1jb3ZpZC0xOS1jYXNlcy1mcmlkYXktYWRkcy0zLWRlYXRocy8yMzg3OTAwLz9hbXA?oc=5')